# Preparing SVS Slides


In [ ]:
#install libraries
!apt update && apt install -y openslide-tools
!pip install openslide-python
import openslide
from openslide.deepzoom import DeepZoomGenerator
from matplotlib import pyplot as plt
import numpy as np
from skimage import io
import shutil 

In [ ]:
#connect to google drive
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
import os
#get path to where the SVS slides are stored
path= "/content/drive/My Drive/TCGA data/EGFR/exon 21/"
os.chdir(path)
os.getcwd()
q =!ls

In [ ]:
#recommed seperating the process into a few times 
#to prevent timeout and too much files
k= q[:10]

In [ ]:
#read slide file with openslide
slide= [None]*len(k);
paths= [None]*len(k);
for i in range(len(k)):
  paths[i]= path+ k[i]
  slide[i]= openslide.open_slide(paths[i])

In [ ]:
#progress bar
from tqdm import tqdm

#ignore the low-resolution warning
import warnings
warnings.filterwarnings('ignore')

# Tiling SVS Slides

In [ ]:
print("Tiling", len(k), "files")
for i in range(len(k)):
  #tile images into different layers
  data_gen = DeepZoomGenerator(slide[i], tile_size=512, overlap=0, limit_bounds=False)
  #get the level with 20x magnification
  #for 40x magnification, use data_gen.level_count-1
  level = data_gen.level_count-2
  #get dimension at that level
  (a,b)=data_gen.level_dimensions[level]
  result_path= '/tmp/exon/'+ k[i][:23]
  #calculate number of slides
  num_w = int(np.floor(a/512))+1
  num_h = int(np.floor(b/512))+1 

  if not os.path.exists(result_path):
    os.mkdir(result_path)

  print("current image is", k[i][:23])
  print("dimension is",num_w, num_h)

  with tqdm(total=num_w*num_h) as pbar:
    for w in range(num_w):
      for h in range(num_h):
        img = np.array(data_gen.get_tile(level, (w, h))) #切图
        pbar.update(1)
        #if the image is not mostly backgroud
        white =np.sum(np.any(img >= [220, 220, 220], axis=-1))/(512*512)
        if (white<0.5 and (img.shape[0]+ img.shape[1]==1024)):
          io.imsave(os.path.join(result_path, str(w)+'_'+str(h)+".jpg"), img)